In [1]:
import re
import os
vow = 'aȃáiîíuúoóôeéêę'
cons = 'bcdðfghjklmnprstvwxz'

Предобработка.

In [34]:
def get_names(directory):
    name_text = []
    for filename in os.listdir(directory):
        with open(directory + filename, "r", encoding="utf-8") as f:
            a = []
            filename = filename.replace('.txt', '')
            a.append(filename)
            a.append(clean_text(f.read()))
            name_text.append(a)
    return name_text

In [18]:
def clean_text(text):
    new_text = re.sub("\n", " ", text)
    new_text = re.sub("<.+?>.+?</.+?>", "", new_text)
    new_text = re.sub(r'[^\w\s]','',new_text)
    new_text = re.sub(r'[\d]',' ',new_text)
    new_text = new_text.lower()
    for word in new_text.split(' '):
        if len(word)==1:
            new_text = new_text.replace(word, '')
    new_text = re.sub('  ','',new_text)
    return new_text.rstrip().split(' ')

In [35]:
direct = 'C:\\Users\\1\\Desktop\\Kidorashi\\master\\texts\\'
name_text = get_names(direct) #список формата [[название, текст], [название, текст]]

In [25]:
def lists():
    global vow, cons
    vow = list(vow)
    cons = list(cons)
    return

In [38]:
def word_begin(word, name):
    if word.startswith('th') and name == 'tatian':
        word = re.sub('^th', 'd', word)
    elif word.startswith('hl'):
        word = re.sub('^hl', 'l', word)
    elif word.startswith('hn'):
        word = re.sub('^hn', 'n', word)
    elif word.startswith('hr'):
        word = re.sub('^hr', 'r', word)
    elif word.startswith('ch') and word[2] == 'i' and name == 'isidor':
        word = re.sub('^ch', 'g', word)
    elif word.startswith('hu') and word[2] in vow:
        word = re.sub('^hu', 'w', word)
    elif word.startswith('z'):
        word = re.sub('^z', 'z', word)
    elif word.startswith('ch'):
        word = re.sub('^ch', 'k', word)
    elif word.startswith('kh'):
        word = re.sub('^kh', 'k', word)
    elif word.startswith('h'):
        word = re.sub(r"h(?=("+'|'.join(vow)+r"))", "h", word)
    else:
        return word
    return word

In [40]:
def word_any(word, name):
    word = re.sub('dh', 'd', word)
    if name == 'hildebrandslied':
        word = re.sub('ð', 'd', word)
    if 'isidor' in name:
        word = re.sub('gh', 'g', word)
        word = re.sub('tz', 'zz', word)
    word = re.sub('pph', 'pf', word)
    word = re.sub('sc', 'sk', word)
    word = re.sub('sch', 'sk', word)
    word = re.sub('kch', 'k', word)
    word = re.sub('qu', 'kw', word)
    word = re.sub('quh', 'kw', word)
    word = re.sub('(?<!l|p|r)ph', 'pf', word)
    word = re.sub('v(?!v)', 'f', word)
    word = re.sub('c(?!h|e|i)', 'k', word)
    return word

In [41]:
def word_vowel(word, name):
    if name == 'isidor':
        word = re.sub(r"(?<=("+'|'.join(vow)+r"))zss", "ȥȥ", word)
    word = re.sub(r"(?<=("+'|'.join(vow)+r"))c(?=(e|i))", "z", word)
    word = re.sub(r"(?<=("+'|'.join(vow)+r"))c(?!(" + '|'.join(cons) +r"))", "z", word) # РАБОТАЕТ
    word = re.sub(r"(?<=("+'|'.join(vow)+r"))zss^\\b", "ȥȥ", word)
    word = re.sub(r"(?<=("+'|'.join(vow)+r"))zss\\b", "ȥ", word)
    return word

In [31]:
def word_consonant(word):
    word = re.sub(r"(?<=("+'|'.join(cons)+r"))zz", "z", word)
    word = re.sub('(?<=l|r)ph', 'f', word)
    if 'cch' and 'kch' and 'sch' not in word:
        word = re.sub(r"(?<=("+'|'.join(cons)+r"))ch", "k", word)
    if 'skh' not in word:
        word = re.sub(r"(?<=("+'|'.join(cons)+r"))kh", "k", word)
    return word

In [32]:
def word_intervoc(word):
    word = re.sub(r"(?<=("+'|'.join(vow)+r"))zc(?!(" + '|'.join(vow) +r"))", "ȥȥ", word)
    word = re.sub(r"(?<=("+'|'.join(vow)+r"))h(?!(" + '|'.join(vow) +r"))", "hȥ", word)
    return word

In [33]:
def word_work(word, name):
    word = word_begin(word, name)
    word = word_any(word, name)
    word = word_vowel(word, name)
    word = word_consonant(word)
    word = word_intervoc(word)
    return(word)

In [57]:
def add_word(word, new_word, dict_words):
    if new_word in dict_words and word not in dict_words[new_word]:
        dict_words[new_word].append(word)
    else:
        dict_words[new_word] = [word]
    return dict_words

In [58]:
dict_words = {}

for text in name_text:
    for word in text[1]:
        new_word = word_work(word, text[0])
        dict_words = add_word(word, new_word, dict_words)